<a href="https://colab.research.google.com/github/MNoichl/tttms_public/blob/main/converting_biorxiv_formulas_to_LaTeX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install necessary packages

In [ ]:
!pip install compress-pickle
!pip install imagesize

# Load packages

In [ ]:
# parse arxiv data into formulas -> images,
# and fulltexts, and combine into one pickle-dataframe
import tqdm
import pandas as pd
import re
import os
import sys
from bs4 import BeautifulSoup
import imagesize


import subprocess
import platform
import shutil

import numpy as np
from compress_pickle import dump, load
import zipfile

import seaborn as sns
import matplotlib.pyplot as plt
from PIL import Image
import ast



from IPython.display import display, Math
import json
import sys
import base64
import requests
from shutil import copyfile
import time



def unzip(source,target):
  with zipfile.ZipFile(source, 'r') as zip_ref:
    zip_ref.extractall(target)


def rm_make(dir):
  try:
    shutil.rmtree(dir)
  except:
    pass

  try:
    !mkdir $dir
  except:
    pass

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Load data

In [ ]:
!gdown --fuzzy https://drive.google.com/file/d/1pKVIs4nrQ4HHv-WYDsBpbrbOIzM1GiRT/view?usp=sharing -O 'already_parsed.zip'
!unzip already_parsed.zip

In [ ]:
def convert(file_path):
    """ Calls mathpix API on a png-file, returns API-json"""
    image_uri = "data:image/gif;base64," + base64.b64encode(open(file_path, "rb").read()).decode()
    
    try:
        image_uri = "data:image/gif;base64," + base64.b64encode(open(file_path, "rb").read()).decode()
    except:
        return 'Error'
    
    r = requests.post("https://api.mathpix.com/v3/latex",
        data=json.dumps({'src': image_uri,
                         'formats': ['latex_styled','latex_normal','text','latex_list','mathml','asciimath','wolfram'],
                         'confidence_threshold' : 0,
                         'ocr':["math", "text"],
                         'beam_size':5,
                         'format_options': {"latex_styled": {"transforms": ["rm_spaces"]}},


                         'skip_recrop': False
                        }),
        headers={"app_id": "app_id", # anonymized for review
                 "app_key": "app_key",
                 "Content-type": "application/json"})
    try:
        return json.loads(r.text)
    except:
        return 'Error'

In [ ]:
full_data_biorxiv = load("drive/MyDrive/biorxiv_zips/biorxiv_mathpixed_zips/biorxiv_frame_up_to_13.bz")

In [ ]:
base_dir = "drive/MyDrive/biorxiv_zips/biorxiv_fulltext_zip_files/"
os.listdir(base_dir)

In [ ]:
parsed_earlier = ['January_2019.zip',
 'September_2019.zip',
 'January_2020.zip',
 'October_2019.zip',
 'April_2019.zip',
 'December_2019.zip',
 'October_2020.zip',
 'February_2020.zip',
 'August_2019.zip',
 'June_2019.zip',
 'July_2019.zip',
 'May_2019.zip',
 'April_2020.zip',
 'May_2020.zip',
 'December_2020.zip']

In [ ]:
this_batch = [x for x in os.listdir(base_dir) if x not in parsed_earlier] # because we had to split processing over several steps
this_batch

In [ ]:

with_mathpix = full_data_biorxiv[full_data_biorxiv['filtered_formulas'] != 'no formulas']
with_mathpix.iloc[np.where(np.array([len(x) for x in with_mathpix['mathpix_returns']]) > 0)]
# with_mathpix
# np.sum([len(x) for x in with_mathpix['mathpix_returns']])

In [ ]:




full_data_biorxiv['formula_images'] = np.empty((len(full_data_biorxiv), 0)).tolist()
full_data_biorxiv['image_sizes'] = np.empty((len(full_data_biorxiv), 0)).tolist()
full_data_biorxiv['formula_images'] = full_data_biorxiv['formula_images'].astype('object')
full_data_biorxiv['image_sizes'] = full_data_biorxiv['image_sizes'].astype('object')



all_files_counter = 0

no_formulas = 20
for this_zip_index,this_zip in tqdm.tqdm_notebook(enumerate(this_batch)): # loop over zip files
  rm_make('target')
  unzip(base_dir + this_zip,'target')
  for article in tqdm.tqdm_notebook(os.listdir('target')): # loop over directories in each batch
    this_articles_formula_images = []
    sizes = []
    for this_file in os.listdir('target/'+article): # extract this articles id
      if this_file.endswith(".xml"):
        article_id = this_file.replace('.xml','')
    w = np.where(full_data_biorxiv['small_id'] == article_id)[0]
    if len(w) == 0: # check if there are formulas
      pass
    else:
      if full_data_biorxiv.iloc[w[-1]]['already_processed']: 
        
        #check whether we already parsed some of the formulas. If yes, integrate them into the dataset
        already_converted_filenames = list(already_parsed_frame[already_parsed_frame['article_id_x']==article_id]['filenames'])
        if len(already_converted_filenames) < no_formulas:
          formulas_w = np.array(range(0,len(already_converted_filenames)))
        else:
          formulas_w = np.random.randint(0,len(already_converted_filenames),no_formulas)

        filenames_to_use = [already_converted_filenames[x] for x in formulas_w]
        article_tuples = []
        article_latex = []
        for this_formula_filename in filenames_to_use:
          with open('biorxiv_tuples_batch/'+this_formula_filename, "r",encoding='utf-8') as text_file:
            tuple_list = text_file.read()
          article_tuples.append(ast.literal_eval(tuple_list))
          with open('expanded_latex_formulas/'+this_formula_filename, "r",encoding='utf-8') as text_file:
            formula = text_file.read()
          article_latex.append(formula)
        full_data_biorxiv.at[w[-1],'formula_images'] = filenames_to_use
        full_data_biorxiv.at[w[-1],'formula_tuples'] = article_tuples
        full_data_biorxiv.at[w[-1],'filtered_formulas'] = article_latex



      else:

        # if the formulas have not yet been converted, convert them using mathpix:
        formula_gifs = []
        for this_file in os.listdir('target/'+article):
          if this_file.endswith(".gif"):
            formula_gifs.append(this_file)
        len_formulas = len(formula_gifs)

        if len_formulas < no_formulas:
          formulas_w = np.array(range(0,len_formulas))
        else:
          formulas_w = np.random.randint(0,len_formulas,no_formulas)

        filenames_to_use = [formula_gifs[x] for x in formulas_w]

        pngs = []
        for this_file in filenames_to_use: # extract this articles id
          size = imagesize.get('target/'+article+'/'+this_file)
          if (size[0] > 200) and (size[0] < 800) and (size[1] < 100) and (size[1] > 40):
            png_name = this_file.replace('.gif','.png')
            im = Image.open('target/'+article+'/'+this_file)
            im.save('target/'+article+'/'+png_name,'PNG')
            pngs.append(png_name)

        mathpix_returns = []
        gathered_latex = []
        for png_name in pngs:
          returned_object = convert('target/'+article+'/'+png_name)
          time.sleep(0.33)
          mathpix_returns.append(returned_object)
          try:
            gathered_latex.append(returned_object['latex_styled'])
          except KeyError:
                print('parsing unsuccesful -error')
          
        
        full_data_biorxiv.at[w[-1],'mathpix_returns'] = mathpix_returns
        full_data_biorxiv.at[w[-1],'filtered_formulas'] = gathered_latex
  dump(full_data_biorxiv, "drive/MyDrive/biorxiv_zips/biorxiv_mathpixed_zips/biorxiv_frame_up_to_"+str(this_zip_index+15)+ ".bz")


